<a href="https://colab.research.google.com/github/florenciasvriz/POD/blob/main/Copia_de_POD_04_Validaci%C3%B3n_de_datos_Individual_TEMPLATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   **Año:** 2024
*   **Alumno/a:** Florencia Svriz
*   **Legajo:** 1008832

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [1]:
import pydantic as pyd

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [2]:
from pydantic import BaseModel
class Client(BaseModel):
    dni: int
    nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [3]:
usuario = Client(dni=39755010, nacionalidad='Argentina')
print(usuario)

dni=39755010 nacionalidad='Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [4]:
usuario = Client(dni='39755010', nacionalidad='Argentina')

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [5]:
!pip install pycountry


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.0 MB/s eta 0:00:00


In [6]:
from typing_extensions import Annotated
from pydantic import BaseModel, Field, validator
from datetime import date
import pycountry

class Client(BaseModel):
    dni: Annotated[int, pyd.Field(ge=0)]
    nacionalidad: Annotated[str, pyd.Field(pattern=r'^[a-zA-Z]+$')]
    fecha_registro: Annotated[date, pyd.Field(le=date.today())]


In [7]:
usuario = Client(dni=39755010, nacionalidad='Argentina', fecha_registro=date(2023, 1, 1))
print(usuario)

dni=39755010 nacionalidad='Argentina' fecha_registro=datetime.date(2023, 1, 1)


In [8]:
usuario = Client(dni=39755010, nacionalidad='Argentina', fecha_registro=date(2025,3,6))


ValidationError: 1 validation error for Client
fecha_registro
  Input should be less than or equal to 2024-11-22 [type=less_than_equal, input_value=datetime.date(2025, 3, 6), input_type=date]
    For further information visit https://errors.pydantic.dev/2.9/v/less_than_equal

In [9]:
usuario = Client(dni=-34562789, nacionalidad='Argentin', fecha_registro=date(2023, 1, 1))

ValidationError: 1 validation error for Client
dni
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-34562789, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than_equal

# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [10]:
!pip install pandera


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 9.8 MB/s eta 0:00:00


In [11]:
import pandas as pd
import pandera as pa
from datetime import date
import pycountry


ClientSchema = pa.DataFrameSchema(
    {
        "dni": pa.Column(int, checks=pa.Check.greater_than_or_equal_to(0)),
        "nacionalidad": pa.Column(str, checks=pa.Check.str_matches(r'^[a-zA-Z ]+$')),
        "fecha_registro": pa.Column(pa.Date, checks=pa.Check.le(date.today())),
    },
    strict=True,
)



In [12]:
data = {
    "dni": [39755010],
    "nacionalidad": ["Brasil"],
    "fecha_registro": [date(2023, 1, 1)]
}

clientes_df = pd.DataFrame(data)

try:
    validated_df = ClientSchema.validate(clientes_df, lazy=True)
    print(validated_df)
except pa.errors.SchemaErrors as err:
    print(err)


        dni nacionalidad fecha_registro
0  39755010       Brasil     2023-01-01


In [13]:
data = {
    "dni": [-39755010],
    "nacionalidad": ["Argentina"],
    "fecha_registro": [date(2023, 1, 1)]
}

clientes_df = pd.DataFrame(data)

try:
    validated_df = ClientSchema.validate(clientes_df, lazy=True)
except pa.errors.SchemaErrors as err:
    print(err)

{
    "DATA": {
        "DATAFRAME_CHECK": [
            {
                "schema": null,
                "column": "dni",
                "check": "greater_than_or_equal_to(0)",
                "error": "Column 'dni' failed element-wise validator number 0: greater_than_or_equal_to(0) failure cases: -39755010"
            }
        ]
    }
}


In [14]:
data = {
    "dni": [12345678],
    "nacionalidad": ["Argentina"],
    "fecha_registro": [date(2025, 1, 1)]
}

clientes_df = pd.DataFrame(data)

try:
    validated_df = ClientSchema.validate(clientes_df, lazy=True)
except pa.errors.SchemaErrors as err:
    print(err)

{
    "DATA": {
        "DATAFRAME_CHECK": [
            {
                "schema": null,
                "column": "fecha_registro",
                "check": "less_than_or_equal_to(2024-11-22)",
                "error": "Column 'fecha_registro' failed element-wise validator number 0: less_than_or_equal_to(2024-11-22) failure cases: 2025-01-01"
            }
        ]
    }
}
